In [1]:
import os
import json

# Get best checkpoint epoch
official_dir = 'out/official/official_base'

best_epoch = 30
for fn in os.listdir(os.path.join(official_dir, 'checkpoints')):
    if 'best-checkpoint' in fn:
        best_epoch = int(fn.split('-')[-1].split('.')[0])
print(f'Best epoch: {best_epoch}')

stats = json.load(open(os.path.join(official_dir, 'stats', 'stats.json')))
base_sup_best_epoch_stats = stats[best_epoch-1]

Best epoch: 29


In [2]:
holdout_info = json.load(open('out/holdout_info.json'))
holdout_batches = [list(x.keys()) for x in holdout_info]
holdout_batches

[['P570', 'P571', 'P569', 'P69', 'P54'],
 ['P264', 'P400', 'P577', 'P3373', 'P26'],
 ['P463', 'P361', 'P175', 'P19', 'P495']]

In [3]:
def get_best_epoch_stats(dir):
    best_epoch = 30
    for fn in os.listdir(os.path.join(dir, 'checkpoints')):
        if 'best-checkpoint' in fn:
            best_epoch = int(fn.split('-')[-1].split('.')[0])
    print(f'Best epoch: {best_epoch}')

    stats = json.load(open(os.path.join(dir, 'stats', 'stats.json')))
    return stats[best_epoch-1]

best_stats_holdout0 = get_best_epoch_stats('out/holdout-batch-0/contr-cand_luke-base_tmp-0.01_supw-0.5_embed-768_norm-True/contr-clust_luke-base_tmp-0.01_supw-0.5_embed-768_norm-True/official_luke-base')
best_stats_holdout1 = get_best_epoch_stats('out/holdout-batch-1/contr-cand_luke-base_tmp-0.01_supw-0.5_embed-768_norm-True/contr-clust_luke-base_tmp-0.01_supw-0.5_embed-768_norm-True/official_luke-base')
best_stats_holdout2 = get_best_epoch_stats('out/holdout-batch-2/contr-cand_luke-base_tmp-0.01_supw-0.5_embed-768_norm-True/contr-clust_luke-base_tmp-0.01_supw-0.5_embed-768_norm-True/official_luke-base')

Best epoch: 30
Best epoch: 30
Best epoch: 27


In [4]:
from pprint import pprint
def get_batch_stats(epoch_stats, holdout_batches, batch=None):
    for i, rel_batch in enumerate(holdout_batches):
        if batch is not None and i != batch:
            continue

        print(f"BATCH {i}")
        holdout_ct = 0
        other_ct = 0

        holdout_macro_f1 = 0
        other_macro_f1 = 0

        holdout_macro_prec = 0
        other_macro_prec = 0

        holdout_macro_rec = 0
        other_macro_rec = 0

        for rel, s in epoch_stats['detailed_stats']['rel_stats'].items():
            if rel in rel_batch:
                holdout_ct += 1
                holdout_macro_f1 += s['f1']
                holdout_macro_prec += s['precision']
                holdout_macro_rec += s['recall']

                print(rel, s['f1'], s['precision'], s['recall'])
            else:
                other_ct += 1
                other_macro_f1 += s['f1']
                other_macro_prec += s['precision']
                other_macro_rec += s['recall']

        holdout_macro_f1 /= holdout_ct
        other_macro_f1 /= other_ct

        holdout_macro_prec /= holdout_ct
        other_macro_prec /= other_ct

        holdout_macro_rec /= holdout_ct
        other_macro_rec /= other_ct

        print(f"holdout_macro_f1: {holdout_macro_f1}")
        print(f"other_macro_f1: {other_macro_f1}")

        print(f"holdout_macro_prec: {holdout_macro_prec}")
        print(f"other_macro_prec: {other_macro_prec}")

        print(f"holdout_macro_rec: {holdout_macro_rec}")
        print(f"other_macro_rec: {other_macro_rec}")

print("-=-=-=BASE MODEL=-=-=-=-")
get_batch_stats(base_sup_best_epoch_stats, holdout_batches)
print("-=-=-=MODEL HOLDOUT 0=-=-=-=-")
get_batch_stats(best_stats_holdout0, holdout_batches, batch=0)
print("-=-=-=MODEL HOLDOUT 1=-=-=-=-")
get_batch_stats(best_stats_holdout1, holdout_batches, batch=1)
print("-=-=-=MODEL HOLDOUT 2=-=-=-=-")
get_batch_stats(best_stats_holdout2, holdout_batches, batch=2)

-=-=-=BASE MODEL=-=-=-=-
BATCH 0
P570 0.8994307350090196 0.8713235293797307 0.9294117646694348
P571 0.6013985963861314 0.6515151514657943 0.558441558405296
P569 0.9284671482575737 0.9298245613763209 0.9271137025968771
P54 0.80701753881673 0.7840909090463585 0.8313253011547393
P69 0.7281105941268663 0.63199999994944 0.8586956520805766
holdout_macro_f1: 0.7928849225192642
other_macro_f1: 0.3832679050728327
holdout_macro_prec: 0.773750830243529
other_macro_prec: 0.44803330502824507
holdout_macro_rec: 0.8209975957813848
other_macro_rec: 0.35904588029062867
BATCH 1
P3373 0.5522388009289375 0.5522388059289374 0.5522388059289374
P264 0.643990924704007 0.6960784313384275 0.599156118118179
P26 0.4150943345753729 0.42857142851576996 0.402439024341166
P577 0.7845036269437001 0.771428571410204 0.7980295566305904
P400 0.6478873188563778 0.6301369862150498 0.6666666665700484
holdout_macro_f1: 0.608743001201679
other_macro_f1: 0.3933855930573154
holdout_macro_prec: 0.6156908446816778
other_macro_prec